📘 1️⃣ Project Introduction

# 🎬 AI-Powered Script Generation Web App (ScriptCraft)

This notebook builds a complete AI system that creates:

- Fully structured scripts with scenes and dialogues
- Automatic scene detection and formatting
- Support for genre, theme, characters, setting, and length
- A fast Falcon-1B-Instruct model (4-bit optimized)
- A beautiful multi-page Flask web application
- Script viewer, download, copy, and regenerate features
- Loading animations, tooltips, and a premium UI experience

The notebook installs dependencies, removes tokens for safety,
loads the model lazily for speed, builds the backend logic,
formats the LLM output into structured screenplay format,
and launches the full web app through ngrok.


📘 2️⃣ Install Dependencies

This step installs:

- Flask → backend web framework  
- pyngrok → public sharing URL  
- transformers → model loading  
- accelerate + bitsandbytes → 4-bit quantization  
- Hugging Face login support  
- Folder creation for templates, static, uploads  

No user edits are needed in this cell.


In [ ]:
# ===============================
# 1️⃣ Install dependencies
# ===============================
!pip install flask pyngrok transformers accelerate bitsandbytes
!mkdir -p templates static uploads

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 14.9 MB/s eta 0:00:00


📘 3️⃣ Hugging Face Authentication

# Hugging Face Login (User Needed)

The project uses:  
tiiuae/Falcon3-1B-Instruct

Your HF token has been removed for security.

To generate your own token:
1. Go to: https://huggingface.co/settings/tokens  
2. Create new token  
3. Set permission → Read  
4. Insert your token in the login() function:

login(token="YOUR_HF_TOKEN_HERE")

⚠️ Never share or upload your token publicly.


In [ ]:
# ===============================
# 2️⃣ Hugging Face Authentication
# ===============================
from huggingface_hub import login
login(token="YOUR_HF_TOKEN_HERE")  # 🔑 replace with your HF token


📘 4️⃣ Backend Architecture

This section creates the entire backend logic:

- Loads Falcon model lazily using LRU caching  
- Generates scripts using text-generation pipeline  
- Cleans unwanted system text and prompt artifacts  
- Extracts title, scenes, dialogues, and stage directions  
- Converts raw AI text into structured JSON  
- Sends formatted output to frontend  

The backend supports:
- Genre, theme, characters, setting, duration  
- Long/medium/short generation length  
- 4-bit inference for fast performance  


In [ ]:
%%writefile app.py
from flask import Flask, render_template, request, jsonify
import functools, os, torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, BitsAndBytesConfig
import re

app = Flask(__name__)

# Lazy-load Falcon model
@functools.lru_cache(maxsize=1)
def get_generator():
    MODEL_NAME = "tiiuae/Falcon3-1B-Instruct"

    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_compute_dtype=torch.float16,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4"
    )

    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_auth_token=True)
    model = AutoModelForCausalLM.from_pretrained(
        MODEL_NAME,
        device_map="auto",
        trust_remote_code=True,
        quantization_config=bnb_config,
        torch_dtype=torch.float16,
        use_auth_token=True
    )

    generator = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        max_new_tokens=1200,
        temperature=0.8,
        top_p=0.9
    )
    return generator

# Enhanced script formatting with better structure
def format_script(text):
    # Remove the entire prompt section - everything before "Title:" or first scene
    script_start_patterns = [
        r'Title:\s*',
        r'(?:INT\.|EXT\.)\s+',
        r'FADE IN:',
        r'Scene 1'
    ]

    # Find where the actual script begins
    start_pos = 0
    for pattern in script_start_patterns:
        match = re.search(pattern, text, re.IGNORECASE)
        if match:
            start_pos = match.start()
            break

    # If we found a starting point, cut everything before it
    if start_pos > 0:
        text = text[start_pos:]

    # Also remove common prompt artifacts
    text = re.sub(r'You are a professional scriptwriter.*?(?=Title:|INT\.|EXT\.|Scene)', '', text, flags=re.DOTALL | re.IGNORECASE)
    text = re.sub(r'Theme:.*?(?=Title:|INT\.|EXT\.|Scene)', '', text, flags=re.DOTALL | re.IGNORECASE)
    text = re.sub(r'Characters:.*?(?=Title:|INT\.|EXT\.|Scene)', '', text, flags=re.DOTALL | re.IGNORECASE)
    text = re.sub(r'Create a script with:.*?(?=Title:|INT\.|EXT\.|Scene)', '', text, flags=re.DOTALL | re.IGNORECASE)
    text = re.sub(r'Start writing.*?(?=Title:|INT\.|EXT\.|Scene)', '', text, flags=re.DOTALL | re.IGNORECASE)

    # Extract title if present
    title_match = re.search(r'Title:\s*["\']?(.+?)["\']?(?:\n|INT\.|EXT\.|\()', text, re.IGNORECASE)
    title = title_match.group(1).strip() if title_match else "Untitled Script"

    # Remove the title line from the text so it doesn't appear in scenes
    if title_match:
        text = text[title_match.end():]

    # Split into scenes based on INT./EXT. or scene markers
    scene_pattern = r'((?:INT\.|EXT\.|SCENE)\s+[^\n]+)'
    parts = re.split(scene_pattern, text, flags=re.IGNORECASE)

    scenes = []
    current_scene = {"title": "", "content": []}

    for i, part in enumerate(parts):
        part = part.strip()
        if not part or part in ['Title:', 'Title', '']:
            continue

        # Check if it's a scene heading
        if re.match(r'(?:INT\.|EXT\.|SCENE)', part, re.IGNORECASE):
            if current_scene["content"]:
                scenes.append(current_scene)
            current_scene = {"title": part, "content": []}
        else:
            # Parse dialogue and stage directions
            lines = part.split('\n')
            for line in lines:
                line = line.strip()
                if not line or line in ['[', ']', '(', ')']:
                    continue

                # Skip lines that look like prompt remnants
                if any(skip in line.lower() for skip in ['create a script', 'character names in caps', 'stage directions', 'natural, engaging']):
                    continue

                # Remove brackets at start/end and clean up
                line = line.lstrip('[').rstrip(']').strip()

                if not line:
                    continue

                # Character name (usually in caps or before colon)
                if re.match(r'^[A-Z][A-Z\s]{1,20}$', line):
                    current_scene["content"].append({
                        "type": "character",
                        "text": line.strip()
                    })
                # Character with dialogue on same line
                elif re.match(r'^([A-Z][A-Za-z\s]+):\s*(.+)$', line):
                    char_match = re.match(r'^([A-Z][A-Za-z\s]+):\s*(.+)$', line)
                    current_scene["content"].append({
                        "type": "character",
                        "text": char_match.group(1).strip()
                    })
                    current_scene["content"].append({
                        "type": "dialogue",
                        "text": char_match.group(2).strip()
                    })
                # Stage directions (in parentheses or brackets)
                elif (line.startswith('(') and line.endswith(')')) or line.startswith('['):
                    direction_text = line.strip('()[]').strip()
                    # Skip common camera directions
                    if not any(cam in direction_text.lower() for cam in ['camera', 'cut to', 'fade', 'zoom', 'end scene']):
                        current_scene["content"].append({
                            "type": "stage_direction",
                            "text": direction_text
                        })
                # Setting description
                elif line.lower().startswith('setting:'):
                    current_scene["content"].append({
                        "type": "setting",
                        "text": line.replace('Setting:', '').replace('setting:', '').strip()
                    })
                # Regular dialogue or description
                elif len(line) > 3 and not line.endswith(':'):
                    current_scene["content"].append({
                        "type": "dialogue",
                        "text": line
                    })

    # Add last scene
    if current_scene["content"]:
        scenes.append(current_scene)

    # If no proper scenes were found, create a simple structure
    if not scenes:
        simple_lines = text.split('\n')
        current_scene = {"title": "Scene 1", "content": []}
        for line in simple_lines:
            line = line.strip()
            if line and len(line) > 2:
                current_scene["content"].append({
                    "type": "dialogue",
                    "text": line
                })
        if current_scene["content"]:
            scenes.append(current_scene)

    return {
        "title": title,
        "scenes": scenes
    }

@app.route("/")
def home():
    return render_template("index.html")

@app.route("/about")
def about():
    return render_template("about.html")

@app.route("/guide")
def guide():
    return render_template("guide.html")

@app.route("/generate", methods=["POST"])
def generate():
    try:
        data = request.get_json()
        genre = data.get("genre", "")
        theme = data.get("theme", "")
        characters = data.get("characters", "")
        setting = data.get("setting", "")
        duration = data.get("duration", "short")

        # Adjust token count based on duration
        token_map = {"short": 800, "medium": 1200, "long": 1600}
        max_tokens = token_map.get(duration, 1200)

        prompt = f"""Write a {genre} script.

Theme: {theme}
Characters: {characters}
Setting: {setting}

Format:
Title: [creative title]
INT. LOCATION - DAY
CHARACTER NAME
(action)
Dialogue here.

Script:"""

        generator = get_generator()
        result = generator(prompt, max_new_tokens=max_tokens)
        script_output = result[0]["generated_text"]

        formatted_script = format_script(script_output)
        return jsonify({"success": True, "script": formatted_script})

    except Exception as e:
        return jsonify({"success": False, "error": str(e)})

if __name__ == "__main__":
    app.run(host="0.0.0.0", port=8000, debug=False)

Overwriting app.py


📘 5️⃣ Frontend Template

The main user interface includes:

- Navigation bar  
- Animated hero section  
- Step-by-step guidance  
- Script generator form  
- Genre selector, theme, characters, setting  
- Script length radio options  
- Loading animation (clapboard icon)  
- Results panel showing formatted script  
- Buttons to copy, download, regenerate  

Stored in:
templates/index.html


In [ ]:
%%writefile templates/index.html
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>🎭 ScriptCraft - AI Script Generator</title>
    <link rel="stylesheet" href="{{ url_for('static', filename='style.css') }}">
    <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/6.4.0/css/all.min.css">
</head>
<body>
    <!-- Navigation -->
    <nav class="navbar">
        <div class="nav-container">
            <div class="logo">
                <i class="fas fa-theater-masks"></i>
                <span>ScriptCraft</span>
            </div>
            <ul class="nav-menu">
                <li><a href="/" class="active">🏠 Home</a></li>
                <li><a href="/guide">📖 Guide</a></li>
                <li><a href="/about">ℹ️ About</a></li>
            </ul>
        </div>
    </nav>

    <!-- Hero Section -->
    <section class="hero">
        <div class="hero-content">
            <h1 class="hero-title">
                <span class="emoji-float">🎬</span>
                Create Amazing Scripts with AI
                <span class="emoji-float">✨</span>
            </h1>
            <p class="hero-subtitle">Transform your ideas into professional scripts in seconds!</p>
            <div class="hero-stats">
                <div class="stat-item">
                    <i class="fas fa-magic"></i>
                    <span>AI-Powered</span>
                </div>
                <div class="stat-item">
                    <i class="fas fa-bolt"></i>
                    <span>Instant Results</span>
                </div>
                <div class="stat-item">
                    <i class="fas fa-smile"></i>
                    <span>Easy to Use</span>
                </div>
            </div>
        </div>
    </section>

    <!-- Main Content -->
    <main class="container">
        <!-- Instructions Card -->
        <div class="instructions-card">
            <h2>🚀 How to Get Started</h2>
            <div class="instruction-steps">
                <div class="step">
                    <div class="step-number">1</div>
                    <div class="step-content">
                        <h3>Choose Your Genre</h3>
                        <p>Select from Comedy, Drama, Horror, Romance, or any genre you like!</p>
                    </div>
                </div>
                <div class="step">
                    <div class="step-number">2</div>
                    <div class="step-content">
                        <h3>Add Details</h3>
                        <p>Enter theme, characters, and setting to personalize your script</p>
                    </div>
                </div>
                <div class="step">
                    <div class="step-number">3</div>
                    <div class="step-content">
                        <h3>Generate & Enjoy</h3>
                        <p>Click generate and watch AI create your script instantly!</p>
                    </div>
                </div>
            </div>
        </div>

        <!-- Generator Form -->
        <div class="generator-card">
            <h2 class="section-title">
                <i class="fas fa-edit"></i> Script Generator
            </h2>

            <form id="scriptForm">
                <div class="form-group">
                    <label for="genre">
                        <i class="fas fa-film"></i> Genre
                        <span class="tooltip">?
                            <span class="tooltiptext">Choose the type of script: Comedy, Drama, Horror, etc.</span>
                        </span>
                    </label>
                    <select id="genre" name="genre" required>
                        <option value="">Select a genre...</option>
                        <option value="Comedy">😂 Comedy</option>
                        <option value="Drama">🎭 Drama</option>
                        <option value="Horror">👻 Horror</option>
                        <option value="Romance">💕 Romance</option>
                        <option value="Action">💥 Action</option>
                        <option value="Mystery">🔍 Mystery</option>
                        <option value="Fantasy">🧙 Fantasy</option>
                        <option value="Sci-Fi">🚀 Sci-Fi</option>
                    </select>
                </div>

                <div class="form-group">
                    <label for="theme">
                        <i class="fas fa-lightbulb"></i> Theme
                        <span class="tooltip">?
                            <span class="tooltiptext">Main idea or message: Friendship, Betrayal, Love, etc.</span>
                        </span>
                    </label>
                    <input type="text" id="theme" name="theme" placeholder="e.g., Friendship, Betrayal, Courage" required>
                </div>

                <div class="form-group">
                    <label for="characters">
                        <i class="fas fa-users"></i> Characters
                        <span class="tooltip">?
                            <span class="tooltiptext">List character names separated by commas</span>
                        </span>
                    </label>
                    <input type="text" id="characters" name="characters" placeholder="e.g., John, Sarah, Mike" required>
                </div>

                <div class="form-group">
                    <label for="setting">
                        <i class="fas fa-map-marker-alt"></i> Setting
                        <span class="tooltip">?
                            <span class="tooltiptext">Where does your story take place?</span>
                        </span>
                    </label>
                    <input type="text" id="setting" name="setting" placeholder="e.g., High School, Coffee Shop, Forest" required>
                </div>

                <div class="form-group">
                    <label for="duration">
                        <i class="fas fa-clock"></i> Script Length
                    </label>
                    <div class="duration-options">
                        <label class="duration-option">
                            <input type="radio" name="duration" value="short" checked>
                            <span>Short (2-3 min)</span>
                        </label>
                        <label class="duration-option">
                            <input type="radio" name="duration" value="medium">
                            <span>Medium (4-5 min)</span>
                        </label>
                        <label class="duration-option">
                            <input type="radio" name="duration" value="long">
                            <span>Long (6-8 min)</span>
                        </label>
                    </div>
                </div>

                <button type="submit" class="btn-generate" id="generateBtn">
                    <i class="fas fa-magic"></i> Generate Script
                </button>
            </form>
        </div>

        <!-- Loading Animation -->
        <div id="loadingSection" class="loading-section" style="display: none;">
            <div class="loading-animation">
                <div class="clapboard">
                    <div class="clap-top"></div>
                    <div class="clap-bottom"></div>
                </div>
                <p class="loading-text">Creating your masterpiece...</p>
                <div class="progress-bar">
                    <div class="progress-fill"></div>
                </div>
            </div>
        </div>

        <!-- Results Section -->
        <div id="resultSection" class="result-section" style="display: none;">
            <div class="result-header">
                <h2 id="scriptTitle">✨ Your Generated Script</h2>
                <div class="result-actions">
                    <button class="btn-action" onclick="downloadScript()">
                        <i class="fas fa-download"></i> Download
                    </button>
                    <button class="btn-action" onclick="copyScript()">
                        <i class="fas fa-copy"></i> Copy
                    </button>
                    <button class="btn-action" onclick="resetForm()">
                        <i class="fas fa-redo"></i> New Script
                    </button>
                </div>
            </div>

            <div id="scriptOutput" class="script-output"></div>
        </div>
    </main>

    <!-- Footer -->
    <footer class="footer">
        <p>Made with ❤️ using AI | © 2024 ScriptCraft</p>
    </footer>

    <script src="{{ url_for('static', filename='script.js') }}"></script>
</body>
</html>

Overwriting templates/index.html


📘 6️⃣ Guide Page

The guide page explains:

- What ScriptCraft is  
- How to generate scripts  
- Genre, theme, character, setting details  
- Tips for best-quality scripts  
- Examples and use cases  
- Step-by-step instructions  

Stored in:
templates/guide.html


In [ ]:
%%writefile templates/guide.html
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Guide - ScriptCraft</title>
    <link rel="stylesheet" href="{{ url_for('static', filename='style.css') }}">
    <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/6.4.0/css/all.min.css">
</head>
<body>
    <nav class="navbar">
        <div class="nav-container">
            <div class="logo">
                <i class="fas fa-theater-masks"></i>
                <span>ScriptCraft</span>
            </div>
            <ul class="nav-menu">
                <li><a href="/">🏠 Home</a></li>
                <li><a href="/guide" class="active">📖 Guide</a></li>
                <li><a href="/about">ℹ️ About</a></li>
            </ul>
        </div>
    </nav>

    <main class="container" style="margin-top: 100px;">
        <div class="guide-page">
            <h1 class="page-title">📖 User Guide</h1>

            <div class="guide-section">
                <h2><i class="fas fa-question-circle"></i> What is ScriptCraft?</h2>
                <p>ScriptCraft is an AI-powered script generator that helps you create professional scripts for skits, dramas, plays, and more. Whether you're a student, teacher, content creator, or theater enthusiast, our tool makes scriptwriting easy and fun!</p>
            </div>

            <div class="guide-section">
                <h2><i class="fas fa-rocket"></i> Getting Started</h2>
                <div class="guide-steps">
                    <div class="guide-step">
                        <h3>Step 1: Choose Your Genre</h3>
                        <p>Select from 8 popular genres including Comedy, Drama, Horror, Romance, Action, Mystery, Fantasy, and Sci-Fi. This sets the tone for your entire script.</p>
                        <div class="example-box">
                            <strong>Example:</strong> Choose "Comedy" for a funny school skit
                        </div>
                    </div>

                    <div class="guide-step">
                        <h3>Step 2: Define Your Theme</h3>
                        <p>What's your story about? Enter a central theme or message like Friendship, Betrayal, Courage, Love, Adventure, etc.</p>
                        <div class="example-box">
                            <strong>Example:</strong> "Friendship overcoming obstacles"
                        </div>
                    </div>

                    <div class="guide-step">
                        <h3>Step 3: Add Characters</h3>
                        <p>List your character names separated by commas. You can have 2-5 characters for best results.</p>
                        <div class="example-box">
                            <strong>Example:</strong> "Emma, Jack, Sophia"
                        </div>
                    </div>

                    <div class="guide-step">
                        <h3>Step 4: Set the Scene</h3>
                        <p>Where does your story take place? Be specific about the location.</p>
                        <div class="example-box">
                            <strong>Example:</strong> "College Library" or "Abandoned Mansion"
                        </div>
                    </div>

                    <div class="guide-step">
                        <h3>Step 5: Choose Length</h3>
                        <p>Select how long you want your script to be:</p>
                        <ul>
                            <li><strong>Short:</strong> 2-3 minutes (perfect for quick skits)</li>
                            <li><strong>Medium:</strong> 4-5 minutes (standard performance length)</li>
                            <li><strong>Long:</strong> 6-8 minutes (detailed storylines)</li>
                        </ul>
                    </div>

                    <div class="guide-step">
                        <h3>Step 6: Generate!</h3>
                        <p>Click the "Generate Script" button and wait 10-30 seconds. Our AI will create a structured script with scenes, dialogues, and stage directions.</p>
                    </div>
                </div>
            </div>

            <div class="guide-section">
                <h2><i class="fas fa-lightbulb"></i> Tips for Best Results</h2>
                <div class="tips-grid">
                    <div class="tip-card">
                        <i class="fas fa-check-circle"></i>
                        <h3>Be Specific</h3>
                        <p>The more specific your inputs, the better your script. Instead of "school," try "high school cafeteria."</p>
                    </div>
                    <div class="tip-card">
                        <i class="fas fa-users"></i>
                        <h3>Character Names</h3>
                        <p>Use clear, distinct names. Avoid similar-sounding names that might confuse readers.</p>
                    </div>
                    <div class="tip-card">
                        <i class="fas fa-palette"></i>
                        <h3>Mix Genres</h3>
                        <p>Don't be afraid to experiment! Try "Comedy-Horror" or "Romance-Mystery" for unique scripts.</p>
                    </div>
                    <div class="tip-card">
                        <i class="fas fa-edit"></i>
                        <h3>Edit & Refine</h3>
                        <p>Use the generated script as a starting point. Feel free to edit and personalize it!</p>
                    </div>
                </div>
            </div>

            <div class="guide-section">
                <h2><i class="fas fa-book"></i> Understanding Your Script</h2>
                <p>Generated scripts include:</p>
                <ul class="feature-list">
                    <li><strong>Scene Headings:</strong> Shows location and time (e.g., INT. CLASSROOM - DAY)</li>
                    <li><strong>Character Names:</strong> Displayed in caps before their dialogue</li>
                    <li><strong>Stage Directions:</strong> Actions and movements in italics or parentheses</li>
                    <li><strong>Dialogue:</strong> What characters say, formatted clearly</li>
                    <li><strong>Scene Transitions:</strong> Smooth flow between different scenes</li>
                </ul>
            </div>

            <div class="guide-section cta-section">
                <h2>🎬 Ready to Create?</h2>
                <p>Now that you know how it works, let's create your first script!</p>
                <a href="/" class="btn-primary">Start Generating</a>
            </div>
        </div>
    </main>

    <footer class="footer">
        <p>Made with ❤️ using AI | © 2024 ScriptCraft</p>
    </footer>
</body>
</html>

Overwriting templates/guide.html


📘 7️⃣ About Page

The about page includes:

- Mission of ScriptCraft  
- Technology behind Falcon and LLMs  
- Features overview  
- Use cases (Education, Theatre, Content Creation)  
- Notes on AI limitations  
- Copyright flexibility  
- Call-to-action section  

Stored in:
templates/about.html


In [ ]:
%%writefile templates/about.html
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>About - ScriptCraft</title>
    <link rel="stylesheet" href="{{ url_for('static', filename='style.css') }}">
    <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/6.4.0/css/all.min.css">
</head>
<body>
    <nav class="navbar">
        <div class="nav-container">
            <div class="logo">
                <i class="fas fa-theater-masks"></i>
                <span>ScriptCraft</span>
            </div>
            <ul class="nav-menu">
                <li><a href="/">🏠 Home</a></li>
                <li><a href="/guide">📖 Guide</a></li>
                <li><a href="/about" class="active">ℹ️ About</a></li>
            </ul>
        </div>
    </nav>

    <main class="container" style="margin-top: 100px;">
        <div class="about-page">
            <h1 class="page-title">ℹ️ About ScriptCraft</h1>

            <div class="about-section">
                <h2>🎭 Our Mission</h2>
                <p>ScriptCraft was created to democratize scriptwriting and make it accessible to everyone. Whether you're a student preparing for a school play, a teacher designing educational activities, a content creator, or just someone with a story to tell, we believe everyone should have access to professional scriptwriting tools.</p>
            </div>

            <div class="about-section">
                <h2>🤖 The Technology</h2>
                <p>ScriptCraft is powered by advanced AI language models that have been trained on thousands of scripts, plays, and dramatic works. Our AI understands:</p>
                <ul class="tech-list">
                    <li><i class="fas fa-brain"></i> Narrative structure and story arcs</li>
                    <li><i class="fas fa-comments"></i> Natural dialogue and character interactions</li>
                    <li><i class="fas fa-theater-masks"></i> Stage directions and dramatic timing</li>
                    <li><i class="fas fa-palette"></i> Genre conventions and storytelling techniques</li>
                </ul>
            </div>

            <div class="about-section">
                <h2>✨ Features</h2>
                <div class="features-grid">
                    <div class="feature-card">
                        <i class="fas fa-bolt"></i>
                        <h3>Lightning Fast</h3>
                        <p>Generate complete scripts in under 30 seconds</p>
                    </div>
                    <div class="feature-card">
                        <i class="fas fa-adjust"></i>
                        <h3>Customizable</h3>
                        <p>Control genre, theme, characters, and setting</p>
                    </div>
                    <div class="feature-card">
                        <i class="fas fa-file-alt"></i>
                        <h3>Professional Format</h3>
                        <p>Industry-standard script formatting</p>
                    </div>
                    <div class="feature-card">
                        <i class="fas fa-download"></i>
                        <h3>Easy Export</h3>
                        <p>Download or copy your scripts instantly</p>
                    </div>
                </div>
            </div>

            <div class="about-section">
                <h2>🎯 Use Cases</h2>
                <div class="use-cases">
                    <div class="use-case">
                        <h3>🎓 Education</h3>
                        <p>Teachers can create scripts for classroom activities, drama clubs, and creative writing assignments.</p>
                    </div>
                    <div class="use-case">
                        <h3>🎬 Content Creation</h3>
                        <p>YouTubers, TikTokers, and content creators can generate scripts for skits and videos.</p>
                    </div>
                    <div class="use-case">
                        <h3>🎭 Theater</h3>
                        <p>Drama groups and theater enthusiasts can quickly prototype scenes and develop ideas.</p>
                    </div>
                    <div class="use-case">
                        <h3>📚 Creative Writing</h3>
                        <p>Writers can overcome creative blocks and explore new story directions.</p>
                    </div>
                </div>
            </div>

            <div class="about-section">
                <h2>🌟 Why Choose ScriptCraft?</h2>
                <div class="why-choose">
                    <div class="reason">
                        <span class="reason-number">1</span>
                        <div>
                            <h3>Free to Use</h3>
                            <p>No hidden costs or subscriptions required</p>
                        </div>
                    </div>
                    <div class="reason">
                        <span class="reason-number">2</span>
                        <div>
                            <h3>No Sign-Up</h3>
                            <p>Start creating immediately without registration</p>
                        </div>
                    </div>
                    <div class="reason">
                        <span class="reason-number">3</span>
                        <div>
                            <h3>Beginner Friendly</h3>
                            <p>Intuitive interface perfect for first-time users</p>
                        </div>
                    </div>
                    <div class="reason">
                        <span class="reason-number">4</span>
                        <div>
                            <h3>Unlimited Generations</h3>
                            <p>Create as many scripts as you need</p>
                        </div>
                    </div>
                </div>
            </div>

            <div class="about-section disclaimer-section">
                <h2>⚠️ Important Notes</h2>
                <div class="disclaimer">
                    <p><strong>AI-Generated Content:</strong> Scripts are created by AI and should be reviewed and edited before use. The AI may occasionally produce inconsistencies or unexpected content.</p>
                    <p><strong>Creative Tool:</strong> ScriptCraft is meant to be a starting point for your creativity, not a replacement for human writers.</p>
                    <p><strong>Copyright:</strong> You are free to use, modify, and distribute the scripts generated for personal, educational, or commercial purposes.</p>
                </div>
            </div>

            <div class="about-section cta-section">
                <h2>🚀 Ready to Start?</h2>
                <p>Join thousands of creators who have brought their stories to life with ScriptCraft!</p>
                <a href="/" class="btn-primary">Create Your First Script</a>
            </div>
        </div>
    </main>

    <footer class="footer">
        <p>Made with ❤️ using AI | © 2024 ScriptCraft</p>
    </footer>
</body>
</html>

Overwriting templates/about.html


📘 8️⃣ CSS Stylesheet

This stylesheet defines:

- Gradient backgrounds  
- Floating emoji animations  
- Navbar with blur effect  
- Card components for generator and instructions  
- Input styling  
- Tooltip interactions  
- Loading animation  
- Scene layout formatting  
- Responsive view for mobile  

Stored in:
static/style.css


In [ ]:
%%writefile static/style.css
@import url('https://fonts.googleapis.com/css2?family=Poppins:wght@300;400;500;600;700&display=swap');

:root {
    --primary: #6366f1;
    --primary-dark: #4f46e5;
    --secondary: #ec4899;
    --success: #10b981;
    --warning: #f59e0b;
    --danger: #ef4444;
    --dark: #1e293b;
    --light: #f8fafc;
    --gray: #64748b;
    --gradient: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
    --shadow: 0 10px 30px rgba(0, 0, 0, 0.1);
    --shadow-lg: 0 20px 50px rgba(0, 0, 0, 0.15);
}

* {
    margin: 0;
    padding: 0;
    box-sizing: border-box;
}

body {
    font-family: 'Poppins', sans-serif;
    background: linear-gradient(135deg, #667eea 0%, #764ba2 50%, #f093fb 100%);
    background-attachment: fixed;
    color: var(--dark);
    line-height: 1.6;
    min-height: 100vh;
}

/* Navbar */
.navbar {
    position: fixed;
    top: 0;
    width: 100%;
    background: rgba(255, 255, 255, 0.95);
    backdrop-filter: blur(10px);
    box-shadow: var(--shadow);
    z-index: 1000;
    animation: slideDown 0.5s ease;
}

@keyframes slideDown {
    from {
        transform: translateY(-100%);
    }
    to {
        transform: translateY(0);
    }
}

.nav-container {
    max-width: 1200px;
    margin: 0 auto;
    padding: 1rem 2rem;
    display: flex;
    justify-content: space-between;
    align-items: center;
}

.logo {
    display: flex;
    align-items: center;
    gap: 0.5rem;
    font-size: 1.5rem;
    font-weight: 700;
    color: var(--primary);
}

.logo i {
    font-size: 1.8rem;
    animation: rotate 3s linear infinite;
}

@keyframes rotate {
    0%, 100% {
        transform: rotate(0deg);
    }
    50% {
        transform: rotate(10deg);
    }
}

.nav-menu {
    display: flex;
    list-style: none;
    gap: 2rem;
}

.nav-menu a {
    text-decoration: none;
    color: var(--dark);
    font-weight: 500;
    transition: all 0.3s ease;
    padding: 0.5rem 1rem;
    border-radius: 8px;
}

.nav-menu a:hover, .nav-menu a.active {
    background: var(--gradient);
    color: white;
    transform: translateY(-2px);
}

/* Hero Section */
.hero {
    margin-top: 80px;
    padding: 4rem 2rem;
    text-align: center;
    animation: fadeIn 1s ease;
}

@keyframes fadeIn {
    from {
        opacity: 0;
        transform: translateY(20px);
    }
    to {
        opacity: 1;
        transform: translateY(0);
    }
}

.hero-content {
    max-width: 800px;
    margin: 0 auto;
}

.hero-title {
    font-size: 3rem;
    font-weight: 700;
    color: white;
    margin-bottom: 1rem;
    text-shadow: 2px 2px 10px rgba(0, 0, 0, 0.2);
}

.emoji-float {
    display: inline-block;
    animation: float 3s ease-in-out infinite;
}

@keyframes float {
    0%, 100% {
        transform: translateY(0px);
    }
    50% {
        transform: translateY(-20px);
    }
}

.hero-subtitle {
    font-size: 1.25rem;
    color: rgba(255, 255, 255, 0.9);
    margin-bottom: 2rem;
}

.hero-stats {
    display: flex;
    justify-content: center;
    gap: 2rem;
    flex-wrap: wrap;
}

.stat-item {
    display: flex;
    flex-direction: column;
    align-items: center;
    gap: 0.5rem;
    padding: 1rem 2rem;
    background: rgba(255, 255, 255, 0.2);
    backdrop-filter: blur(10px);
    border-radius: 12px;
    color: white;
    transition: all 0.3s ease;
}

.stat-item:hover {
    transform: translateY(-5px);
    background: rgba(255, 255, 255, 0.3);
}

.stat-item i {
    font-size: 2rem;
}

/* Container */
.container {
    max-width: 1200px;
    margin: 2rem auto;
    padding: 0 2rem;
}

/* Instructions Card */
.instructions-card {
    background: white;
    border-radius: 20px;
    padding: 2rem;
    margin-bottom: 2rem;
    box-shadow: var(--shadow-lg);
    animation: slideUp 0.6s ease;
}

@keyframes slideUp {
    from {
        opacity: 0;
        transform: translateY(30px);
    }
    to {
        opacity: 1;
        transform: translateY(0);
    }
}

.instructions-card h2 {
    color: var(--primary);
    margin-bottom: 1.5rem;
    font-size: 1.8rem;
}

.instruction-steps {
    display: grid;
    grid-template-columns: repeat(auto-fit, minmax(250px, 1fr));
    gap: 1.5rem;
}

.step {
    display: flex;
    gap: 1rem;
    padding: 1rem;
    background: linear-gradient(135deg, #f8fafc 0%, #e0e7ff 100%);
    border-radius: 12px;
    transition: all 0.3s ease;
}

.step:hover {
    transform: translateY(-5px);
    box-shadow: var(--shadow);
}

.step-number {
    width: 40px;
    height: 40px;
    background: var(--gradient);
    color: white;
    border-radius: 50%;
    display: flex;
    align-items: center;
    justify-content: center;
    font-weight: 700;
    flex-shrink: 0;
}

.step-content h3 {
    color: var(--primary);
    margin-bottom: 0.5rem;
    font-size: 1.1rem;
}

.step-content p {
    color: var(--gray);
    font-size: 0.9rem;
}

/* Generator Card */
.generator-card {
    background: white;
    border-radius: 20px;
    padding: 2.5rem;
    box-shadow: var(--shadow-lg);
    animation: slideUp 0.8s ease;
}

.section-title {
    color: var(--primary);
    margin-bottom: 2rem;
    font-size: 1.8rem;
    display: flex;
    align-items: center;
    gap: 0.5rem;
}

.form-group {
    margin-bottom: 1.5rem;
}

.form-group label {
    display: flex;
    align-items: center;
    gap: 0.5rem;
    font-weight: 600;
    color: var(--dark);
    margin-bottom: 0.5rem;
    font-size: 1rem;
}

.form-group label i {
    color: var(--primary);
}

.tooltip {
    position: relative;
    display: inline-block;
    width: 20px;
    height: 20px;
    background: var(--primary);
    color: white;
    border-radius: 50%;
    text-align: center;
    font-size: 0.8rem;
    cursor: help;
    margin-left: 0.5rem;
}

.tooltiptext {
    visibility: hidden;
    width: 200px;
    background-color: var(--dark);
    color: white;
    text-align: center;
    border-radius: 8px;
    padding: 0.5rem;
    position: absolute;
    z-index: 1;
    bottom: 125%;
    left: 50%;
    margin-left: -100px;
    opacity: 0;
    transition: opacity 0.3s;
    font-size: 0.85rem;
}

.tooltip:hover .tooltiptext {
    visibility: visible;
    opacity: 1;
}

input[type="text"], select {
    width: 100%;
    padding: 0.875rem;
    border: 2px solid #e2e8f0;
    border-radius: 12px;
    font-size: 1rem;
    transition: all 0.3s ease;
    font-family: 'Poppins', sans-serif;
}

input[type="text"]:focus, select:focus {
    outline: none;
    border-color: var(--primary);
    box-shadow: 0 0 0 3px rgba(99, 102, 241, 0.1);
}

select {
    cursor: pointer;
    background: white;
}

.duration-options {
    display: flex;
    gap: 1rem;
    flex-wrap: wrap;
}

.duration-option {
    flex: 1;
    min-width: 150px;
    padding: 1rem;
    border: 2px solid #e2e8f0;
    border-radius: 12px;
    cursor: pointer;
    transition: all 0.3s ease;
    display: flex;
    align-items: center;
    gap: 0.5rem;
}

.duration-option:hover {
    border-color: var(--primary);
    background: #f8fafc;
}

.duration-option input[type="radio"] {
    width: auto;
}

.duration-option input[type="radio"]:checked + span {
    color: var(--primary);
    font-weight: 600;
}

.btn-generate {
    width: 100%;
    padding: 1rem 2rem;
    background: var(--gradient);
    color: white;
    border: none;
    border-radius: 12px;
    font-size: 1.1rem;
    font-weight: 600;
    cursor: pointer;
    transition: all 0.3s ease;
    display: flex;
    align-items: center;
    justify-content: center;
    gap: 0.5rem;
    box-shadow: 0 4px 15px rgba(99, 102, 241, 0.4);
}

.btn-generate:hover {
    transform: translateY(-2px);
    box-shadow: 0 6px 20px rgba(99, 102, 241, 0.5);
}

.btn-generate:active {
    transform: translateY(0);
}

/* Loading Section */
.loading-section {
    background: white;
    border-radius: 20px;
    padding: 3rem;
    margin-top: 2rem;
    box-shadow: var(--shadow-lg);
    text-align: center;
}

.loading-animation {
    display: flex;
    flex-direction: column;
    align-items: center;
    gap: 1.5rem;
}

.clapboard {
    width: 100px;
    height: 100px;
    position: relative;
    animation: clap 1.5s ease-in-out infinite;
}

.clap-top {
    width: 100%;
    height: 30px;
    background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
    border-radius: 8px 8px 0 0;
    position: absolute;
    top: 0;
    transform-origin: bottom;
}

.clap-bottom {
    width: 100%;
    height: 70px;
    background: #f8fafc;
    border: 3px solid var(--primary);
    border-radius: 0 0 8px 8px;
    position: absolute;
    bottom: 0;
}

@keyframes clap {
    0%, 100% {
        transform: rotate(0deg);
    }
    50% {
        transform: rotate(-20deg);
    }
}

.loading-text {
    font-size: 1.2rem;
    color: var(--primary);
    font-weight: 600;
}

.progress-bar {
    width: 100%;
    max-width: 400px;
    height: 8px;
    background: #e2e8f0;
    border-radius: 10px;
    overflow: hidden;
}

.progress-fill {
    height: 100%;
    background: var(--gradient);
    animation: progress 2s ease-in-out infinite;
}

@keyframes progress {
    0% {
        width: 0%;
    }
    50% {
        width: 70%;
    }
    100% {
        width: 100%;
    }
}

/* Result Section */
.result-section {
    background: white;
    border-radius: 20px;
    padding: 2rem;
    margin-top: 2rem;
    box-shadow: var(--shadow-lg);
    animation: slideUp 0.6s ease;
}

.result-header {
    display: flex;
    justify-content: space-between;
    align-items: center;
    margin-bottom: 2rem;
    flex-wrap: wrap;
    gap: 1rem;
}

.result-header h2 {
    color: var(--primary);
    font-size: 1.8rem;
}

.result-actions {
    display: flex;
    gap: 1rem;
    flex-wrap: wrap;
}

.btn-action {
    padding: 0.75rem 1.5rem;
    background: white;
    color: var(--primary);
    border: 2px solid var(--primary);
    border-radius: 10px;
    font-weight: 600;
    cursor: pointer;
    transition: all 0.3s ease;
    display: flex;
    align-items: center;
    gap: 0.5rem;
}

.btn-action:hover {
    background: var(--primary);
    color: white;
    transform: translateY(-2px);
}

/* Script Output */
.script-output {
    background: #f8fafc;
    border-radius: 12px;
    padding: 2rem;
    max-height: 600px;
    overflow-y: auto;
}

.script-scene {
    margin-bottom: 2rem;
    padding: 1.5rem;
    background: white;
    border-left: 4px solid var(--primary);
    border-radius: 8px;
    animation: fadeIn 0.5s ease;
}

.scene-title {
    font-size: 1.3rem;
    font-weight: 700;
    color: var(--primary);
    margin-bottom: 1rem;
    display: flex;
    align-items: center;
    gap: 0.5rem;
}

.scene-content {
    line-height: 1.8;
}

.character-name {
    font-weight: 700;
    color: var(--secondary);
    margin-top: 1rem;
    margin-bottom: 0.25rem;
    text-transform: uppercase;
    font-size: 0.95rem;
}

.dialogue {
    color: var(--dark);
    margin-left: 1rem;
    margin-bottom: 0.5rem;
}

.stage-direction {
    font-style: italic;
    color: var(--gray);
    margin: 0.5rem 0;
    margin-left: 2rem;
}

.setting {
    background: linear-gradient(135deg, #e0e7ff 0%, #f8fafc 100%);
    padding: 0.75rem;
    border-radius: 8px;
    margin-bottom: 1rem;
    color: var(--dark);
    font-weight: 500;
}

/* Guide & About Pages */
.guide-page, .about-page {
    background: white;
    border-radius: 20px;
    padding: 3rem;
    box-shadow: var(--shadow-lg);
    animation: fadeIn 0.8s ease;
}

.page-title {
    font-size: 2.5rem;
    color: var(--primary);
    margin-bottom: 2rem;
    text-align: center;
}

.guide-section, .about-section {
    margin-bottom: 3rem;
}

.guide-section h2, .about-section h2 {
    color: var(--primary);
    margin-bottom: 1.5rem;
    font-size: 1.8rem;
    display: flex;
    align-items: center;
    gap: 0.5rem;
}

.guide-steps {
    display: flex;
    flex-direction: column;
    gap: 2rem;
}

.guide-step {
    padding: 1.5rem;
    background: linear-gradient(135deg, #f8fafc 0%, #e0e7ff 100%);
    border-radius: 12px;
    border-left: 4px solid var(--primary);
}

.guide-step h3 {
    color: var(--primary);
    margin-bottom: 0.75rem;
}

.example-box {
    margin-top: 1rem;
    padding: 1rem;
    background: white;
    border-radius: 8px;
    border-left: 3px solid var(--secondary);
}

.tips-grid, .features-grid {
    display: grid;
    grid-template-columns: repeat(auto-fit, minmax(250px, 1fr));
    gap: 1.5rem;
    margin-top: 1.5rem;
}

.tip-card, .feature-card {
    padding: 1.5rem;
    background: linear-gradient(135deg, #f8fafc 0%, #e0e7ff 100%);
    border-radius: 12px;
    text-align: center;
    transition: all 0.3s ease;
}

.tip-card:hover, .feature-card:hover {
    transform: translateY(-5px);
    box-shadow: var(--shadow);
}

.tip-card i, .feature-card i {
    font-size: 2.5rem;
    color: var(--primary);
    margin-bottom: 1rem;
}

.feature-list {
    list-style: none;
    margin-top: 1rem;
}

.feature-list li {
    padding: 0.75rem;
    margin-bottom: 0.5rem;
    background: #f8fafc;
    border-radius: 8px;
    border-left: 3px solid var(--primary);
}

.use-cases {
    display: grid;
    grid-template-columns: repeat(auto-fit, minmax(250px, 1fr));
    gap: 1.5rem;
    margin-top: 1.5rem;
}

.use-case {
    padding: 1.5rem;
    background: linear-gradient(135deg, #f8fafc 0%, #e0e7ff 100%);
    border-radius: 12px;
}

.use-case h3 {
    color: var(--primary);
    margin-bottom: 0.75rem;
}

.why-choose {
    display: flex;
    flex-direction: column;
    gap: 1.5rem;
    margin-top: 1.5rem;
}

.reason {
    display: flex;
    align-items: center;
    gap: 1.5rem;
    padding: 1.5rem;
    background: linear-gradient(135deg, #f8fafc 0%, #e0e7ff 100%);
    border-radius: 12px;
}

.reason-number {
    width: 50px;
    height: 50px;
    background: var(--gradient);
    color: white;
    border-radius: 50%;
    display: flex;
    align-items: center;
    justify-content: center;
    font-size: 1.5rem;
    font-weight: 700;
    flex-shrink: 0;
}

.disclaimer-section {
    background: linear-gradient(135deg, #fef3c7 0%, #fde68a 100%);
    padding: 2rem;
    border-radius: 12px;
}

.disclaimer p {
    margin-bottom: 1rem;
}

.cta-section {
    text-align: center;
    padding: 3rem 2rem;
    background: var(--gradient);
    color: white;
    border-radius: 12px;
}

.cta-section h2 {
    color: white;
    margin-bottom: 1rem;
}

.cta-section p {
    font-size: 1.1rem;
    margin-bottom: 2rem;
}

.btn-primary {
    display: inline-block;
    padding: 1rem 2rem;
    background: white;
    color: var(--primary);
    text-decoration: none;
    border-radius: 10px;
    font-weight: 600;
    transition: all 0.3s ease;
    box-shadow: 0 4px 15px rgba(0, 0, 0, 0.2);
}

.btn-primary:hover {
    transform: translateY(-3px);
    box-shadow: 0 6px 20px rgba(0, 0, 0, 0.3);
}

.tech-list {
    list-style: none;
    margin-top: 1rem;
}

.tech-list li {
    padding: 0.75rem;
    margin-bottom: 0.5rem;
    display: flex;
    align-items: center;
    gap: 1rem;
}

.tech-list i {
    color: var(--primary);
    font-size: 1.2rem;
}

/* Footer */
.footer {
    text-align: center;
    padding: 2rem;
    color: white;
    margin-top: 3rem;
}

/* Responsive */
@media (max-width: 768px) {
    .hero-title {
        font-size: 2rem;
    }

    .nav-menu {
        gap: 1rem;
    }

    .nav-menu a {
        padding: 0.5rem;
        font-size: 0.9rem;
    }

    .result-header {
        flex-direction: column;
        align-items: flex-start;
    }

    .duration-options {
        flex-direction: column;
    }

    .guide-page, .about-page {
        padding: 1.5rem;
    }
}

Overwriting static/style.css


📘 9️⃣ JavaScript Logic

The JS file handles:

- Sending user inputs to backend  
- Triggering loading animation  
- Rendering formatted scenes  
- Copying script to clipboard  
- Downloading script as text  
- Notification system with animations  
- Auto-scroll behavior  

Stored in:
static/script.js


In [ ]:
%%writefile static/script.js
// Global variable to store the generated script
let generatedScript = null;

// Form submission handler
document.getElementById('scriptForm').addEventListener('submit', async function(e) {
    e.preventDefault();

    // Get form data
    const formData = {
        genre: document.getElementById('genre').value,
        theme: document.getElementById('theme').value,
        characters: document.getElementById('characters').value,
        setting: document.getElementById('setting').value,
        duration: document.querySelector('input[name="duration"]:checked').value
    };

    // Show loading section
    document.getElementById('loadingSection').style.display = 'block';
    document.getElementById('resultSection').style.display = 'none';

    // Scroll to loading section
    document.getElementById('loadingSection').scrollIntoView({ behavior: 'smooth' });

    try {
        // Send request to backend
        const response = await fetch('/generate', {
            method: 'POST',
            headers: {
                'Content-Type': 'application/json',
            },
            body: JSON.stringify(formData)
        });

        const data = await response.json();

        if (data.success) {
            generatedScript = data.script;
            displayScript(data.script);

            // Hide loading, show results
            setTimeout(() => {
                document.getElementById('loadingSection').style.display = 'none';
                document.getElementById('resultSection').style.display = 'block';
                document.getElementById('resultSection').scrollIntoView({ behavior: 'smooth' });
            }, 1000);
        } else {
            throw new Error(data.error || 'Failed to generate script');
        }
    } catch (error) {
        console.error('Error:', error);
        document.getElementById('loadingSection').style.display = 'none';
        alert('❌ Error generating script: ' + error.message + '\n\nPlease try again!');
    }
});

// Display the generated script with proper formatting
function displayScript(script) {
    const output = document.getElementById('scriptOutput');
    const titleElement = document.getElementById('scriptTitle');

    // Update title
    titleElement.innerHTML = `✨ ${script.title}`;

    // Clear previous content
    output.innerHTML = '';

    // Create scene elements
    script.scenes.forEach((scene, index) => {
        const sceneDiv = document.createElement('div');
        sceneDiv.className = 'script-scene';

        // Scene title
        if (scene.title) {
            const sceneTitle = document.createElement('div');
            sceneTitle.className = 'scene-title';
            sceneTitle.innerHTML = `<i class="fas fa-film"></i> ${scene.title}`;
            sceneDiv.appendChild(sceneTitle);
        }

        // Scene content
        const contentDiv = document.createElement('div');
        contentDiv.className = 'scene-content';

        scene.content.forEach(item => {
            const element = document.createElement('div');

            switch(item.type) {
                case 'character':
                    element.className = 'character-name';
                    element.textContent = item.text;
                    break;
                case 'dialogue':
                    element.className = 'dialogue';
                    element.textContent = item.text;
                    break;
                case 'stage_direction':
                    element.className = 'stage-direction';
                    element.textContent = `(${item.text})`;
                    break;
                case 'setting':
                    element.className = 'setting';
                    element.innerHTML = `<i class="fas fa-map-marker-alt"></i> ${item.text}`;
                    break;
                default:
                    element.className = 'dialogue';
                    element.textContent = item.text;
            }

            contentDiv.appendChild(element);
        });

        sceneDiv.appendChild(contentDiv);
        output.appendChild(sceneDiv);
    });
}

// Download script as text file
function downloadScript() {
    if (!generatedScript) return;

    let scriptText = `${generatedScript.title}\n\n`;
    scriptText += '='.repeat(50) + '\n\n';

    generatedScript.scenes.forEach((scene, index) => {
        scriptText += `${scene.title}\n\n`;

        scene.content.forEach(item => {
            switch(item.type) {
                case 'character':
                    scriptText += `\n${item.text.toUpperCase()}\n`;
                    break;
                case 'dialogue':
                    scriptText += `${item.text}\n`;
                    break;
                case 'stage_direction':
                    scriptText += `(${item.text})\n`;
                    break;
                case 'setting':
                    scriptText += `Setting: ${item.text}\n\n`;
                    break;
            }
        });

        scriptText += '\n' + '-'.repeat(50) + '\n\n';
    });

    // Create download link
    const blob = new Blob([scriptText], { type: 'text/plain' });
    const url = window.URL.createObjectURL(blob);
    const a = document.createElement('a');
    a.href = url;
    a.download = `${generatedScript.title.replace(/[^a-z0-9]/gi, '_')}.txt`;
    document.body.appendChild(a);
    a.click();
    window.URL.revokeObjectURL(url);
    document.body.removeChild(a);

    // Show success message
    showNotification('✅ Script downloaded successfully!');
}

// Copy script to clipboard
function copyScript() {
    if (!generatedScript) return;

    let scriptText = `${generatedScript.title}\n\n`;
    scriptText += '='.repeat(50) + '\n\n';

    generatedScript.scenes.forEach((scene, index) => {
        scriptText += `${scene.title}\n\n`;

        scene.content.forEach(item => {
            switch(item.type) {
                case 'character':
                    scriptText += `\n${item.text.toUpperCase()}\n`;
                    break;
                case 'dialogue':
                    scriptText += `${item.text}\n`;
                    break;
                case 'stage_direction':
                    scriptText += `(${item.text})\n`;
                    break;
                case 'setting':
                    scriptText += `Setting: ${item.text}\n\n`;
                    break;
            }
        });

        scriptText += '\n' + '-'.repeat(50) + '\n\n';
    });

    // Copy to clipboard
    navigator.clipboard.writeText(scriptText).then(() => {
        showNotification('✅ Script copied to clipboard!');
    }).catch(err => {
        console.error('Failed to copy:', err);
        showNotification('❌ Failed to copy script');
    });
}

// Reset form and hide results
function resetForm() {
    document.getElementById('scriptForm').reset();
    document.getElementById('resultSection').style.display = 'none';
    generatedScript = null;

    // Scroll back to form
    document.querySelector('.generator-card').scrollIntoView({ behavior: 'smooth' });
}

// Show notification
function showNotification(message) {
    // Create notification element
    const notification = document.createElement('div');
    notification.style.cssText = `
        position: fixed;
        top: 100px;
        right: 20px;
        background: white;
        padding: 1rem 1.5rem;
        border-radius: 10px;
        box-shadow: 0 10px 30px rgba(0, 0, 0, 0.2);
        z-index: 10000;
        animation: slideIn 0.3s ease;
        font-weight: 600;
        color: #1e293b;
    `;
    notification.textContent = message;

    document.body.appendChild(notification);

    // Remove after 3 seconds
    setTimeout(() => {
        notification.style.animation = 'slideOut 0.3s ease';
        setTimeout(() => {
            document.body.removeChild(notification);
        }, 300);
    }, 3000);
}

// Add CSS animations for notifications
const style = document.createElement('style');
style.textContent = `
    @keyframes slideIn {
        from {
            transform: translateX(400px);
            opacity: 0;
        }
        to {
            transform: translateX(0);
            opacity: 1;
        }
    }

    @keyframes slideOut {
        from {
            transform: translateX(0);
            opacity: 1;
        }
        to {
            transform: translateX(400px);
            opacity: 0;
        }
    }
`;
document.head.appendChild(style);

Overwriting static/script.js


📘 🔟 Kill Previous Processes

This ensures Flask and ngrok do not conflict:

- Stops earlier Flask sessions  
- Stops older ngrok tunnels  
- Prevents "port already in use" errors  

Safe to run every time before starting server.


In [ ]:
# ===============================
# 6️⃣ Kill any previous processes
# ===============================
!pkill -f flask || echo "No flask running"
!pkill -f ngrok || echo "No ngrok running"

^C
^C


📘 1️⃣1️⃣ Checking Port 8000 (User Instructions)

If server fails, port 8000 may be occupied.

Run:
!lsof -i :8000

If you see:
python   12345 LISTEN

Kill it with:
!kill -9 12345

Then launch Flask again.


In [ ]:
!lsof -i :8000

In [ ]:
!kill -9 5759

📘 1️⃣2️⃣ Run Flask App in Background

Starts backend without blocking the notebook:

!nohup python app.py > flask.log 2>&1 &

Logs are stored in flask.log


In [ ]:
# ===============================
# 7️⃣ Run Flask in the background
# ===============================
!nohup python app.py > flask.log 2>&1 &

📘 1️⃣3️⃣ Ngrok Setup

Ngrok provides a public HTTPS link.

Your ngrok token was removed for safety.

To use ngrok:
1. Get token → https://dashboard.ngrok.com/get-started/your-authtoken  
2. Add inside notebook:

conf.get_default().auth_token = "YOUR_NGROK_TOKEN_HERE"

3. Start tunnel:

public_url = ngrok.connect(8000)

Shareable app link appears here.


In [ ]:
# ===============================
# 8️⃣ Start ngrok tunnel
# ===============================
from pyngrok import ngrok, conf
conf.get_default().auth_token = "YOUR_NGROK_TOKEN_HERE" # 🔑 replace with your token

public_url = ngrok.connect(8000)
print("🌍 Public URL:", public_url)

# ===============================
# 9️⃣ Check logs (optional)
# ===============================
!sleep 3 && tail -n 20 flask.log


🌍 Public URL: NgrokTunnel: "https://6c6129b93a84.ngrok-free.app" -> "http://localhost:8000"


📘 1️⃣4️⃣ View Logs

To debug backend:

!tail -n 20 flask.log

Shows:
- Model loading issues  
- Prompt errors  
- Script formatting errors  
- Runtime crashes  


In [ ]:
!tail -n 50 flask.log

2025-11-08 05:16:53.033625: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1762579013.068438    1181 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1762579013.082255    1181 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1762579013.106488    1181 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1762579013.106513    1181 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1762579013.106516    1181 computation_placer.cc:177] computation placer alr

In [ ]:
📘 1️⃣5️⃣ Project Completed

🎉 ScriptCraft Web App is fully ready!

Users can now:

- Enter genre, theme, characters, setting  
- Generate fully structured scripts  
- View scenes and dialogues in a professional layout  
- Download or copy scripts  
- Regenerate new versions instantly  
- Use ngrok link to share publicly  

This project is deployment-ready and portfolio-quality.
